<a href="https://colab.research.google.com/github/aleks-haksly/Postypashka/blob/main/DA/03%20HW%20SQL/HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata

In [2]:
engine = create_engine(userdata.get('supabase'), client_encoding='utf8')

In [3]:
def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

## Задача 1. Для таблиц audiobooks, audio_cards, и listenings

1. вывести сколько пользователей добавили книгу 'Coraline', сколько пользователей прослушало больше 10%.

2. по каждой операционной системе и тайтлу книги вывести кол-во пользователей, сумму прослушивания в часах. не учитывать тестовые прослушивания.

3. найти книгу, которую слушает больше всего людей.

4. найти книгу, которую чаще всего дослушивают до конца.

5. найти среднее время между первым и вторым прослушиванием у пользователей. удалять тестовые чтения не нужно.

6. для каждого тайтла книги вывести трех пользователей с минимальным временем между первым и последним прослушиваниями книги. удалять тестовые чтения нужно. Отсортировать по книге и разнице времени.

In [22]:
# Задание 1
sql = """
WITH t1 AS (
SELECT
	a.uuid,
	a.duration
FROM
	postupashka.audiobooks a
WHERE
	a.title = 'Coraline'
)

SELECT
	count(DISTINCT l.user_id) AS add_count,
	count(DISTINCT l.user_id) FILTER(WHERE l.position_to * 10 > (SELECT t1.duration FROM t1)) AS pct_10_count
FROM
	postupashka.listenings l
WHERE
	l.audiobook_uuid = (SELECT t1.uuid FROM t1)

"""
select(sql)

,add_count,pct_10_count
0,309,49
